In [11]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import json
#!pip install duckdb
import duckdb

In [2]:
ddir = r'/Users/katherinewang/SIGD/My Drive/MAPS/sandbox_data'

In [3]:
con = duckdb.connect()
fe = pq.read_table('{}/freshmen-enrollment.parquet'.format(ddir)).to_pandas()
cb_sest = pq.read_table('{}/census_state_est.parquet'.format(ddir)).to_pandas()

slist = ['Private, 2-year',
 'Private, 4-year or above',
 'Proprietary, 2-year',
 'Proprietary, 4-year or above',
 'Public, 2-year',
 'Public, 4-year or above']

r7map = {'american_indian_or_alaska_native':3,
 'asian':4,
 'black_or_african_american':2,
 'hispanic':7,
 'native_hawaiian_or_other_pacific_islander':5,
 'nonresident_alien':9,
 'race_ethnicity_unknown':8,
 'two_or_more_races':6,
 'white':1}

gmap = {'men':1, 'women':2}

# Corresponds to Census FIPS codes.
st_map = {
'Total':0,
'Alabama':1,
'Alaska':2,
'Arizona':4,
'Arkansas':5,
'California':6,
'Colorado':8,
'Connecticut':9,
'Delaware':10,
'District of Columbia':11,
'Florida':12,
'Georgia':13,
'Hawaii':15,
'Idaho':16,
'Illinois':17,
'Indiana':18,
'Iowa':19,
'Kansas':20,
'Kentucky':21,
'Louisiana':22,
'Maine':23,
'Maryland':24,
'Massachusetts':25,
'Michigan':26,
'Minnesota':27,
'Mississippi':28,
'Missouri':29,
'Montana':30,
'Nebraska':31,
'Nevada':32,
'New Hampshire':33,
'New Jersey':34,
'New Mexico':35,
'New York':36,
'North Carolina':37,
'North Dakota':38,
'Ohio':39,
'Oklahoma':40,
'Oregon':41,
'Pennsylvania':42,
'Rhode Island':44,
'South Carolina':45,
'South Dakota':46,
'Tennessee':47,
'Texas':48,
'Utah':49,
'Vermont':50,
'Virginia':51,
'Washington':53,
'West Virginia':54,
'Wisconsin':55,
'Wyoming':56,
# EFCSTATE specifics
'State unknown':57,
'Outlying areas total':89,
'American Samoa':60,
'Federated States of Micronesia':64,
'Guam':66,
'Marshall Islands':68,
'Northern Marianas':69,
'Palau':70,
'Puerto Rico':72,
'Virgin Islands':78,
'Foreign countries':90,
'Residence not reported':98
}

In [4]:
# ST_FIPS is State where the institution is located
# EFCSTATE is State of residence when student was first admitted - based on the address at time of application
# status indicates if student is IN State or OUT of State (IN <-> ST_FIPS == EFCSTATE)
# EFRES01	First-time degree/certificate-seeking undergraduate students
# EFRES02	First-time degree/certificate-seeking undergraduate students who graduated from high school in the past 12 months
fe

,YEAR,STABBR,ST_FIPS,sector,EFCSTATE,status,efres01,efres02
0,2011,AK,2,"Private, 4-year or above",2,IN,10.0,7.0
1,2011,AK,2,"Private, 4-year or above",6,OUT,1.0,0.0
2,2011,AK,2,"Private, 4-year or above",12,OUT,1.0,1.0
3,2011,AK,2,"Private, 4-year or above",19,OUT,1.0,1.0
4,2011,AK,2,"Private, 4-year or above",27,OUT,1.0,0.0
...,...,...,...,...,...,...,...,...
74939,2020,WY,56,"Public, 4-year or above",53,OUT,19.0,17.0
74940,2020,WY,56,"Public, 4-year or above",55,OUT,8.0,8.0
74941,2020,WY,56,"Public, 4-year or above",56,IN,782.0,732.0
74942,2020,WY,56,"Public, 4-year or above",57,OUT,12.0,10.0


In [5]:
df2019=fe[fe.YEAR==2019]

In [6]:
temp1 = df2019.groupby(['ST_FIPS']).sum()['efres02'].reset_index(name='sum')
temp2 = df2019.groupby(['ST_FIPS','sector','status']).sum()['efres02'].reset_index(name='count')
df_temp = pd.merge(temp1,temp2,on='ST_FIPS')

In [7]:
df_temp

,ST_FIPS,sum,sector,status,count
0,1,32934.0,"Private, 4-year or above",IN,823.0
1,1,32934.0,"Private, 4-year or above",OUT,687.0
2,1,32934.0,"Public, 2-year",IN,10160.0
3,1,32934.0,"Public, 2-year",OUT,537.0
4,1,32934.0,"Public, 4-year or above",IN,12113.0
...,...,...,...,...,...
352,55,34182.0,"Public, 4-year or above",OUT,7907.0
353,56,3080.0,"Public, 2-year",IN,999.0
354,56,3080.0,"Public, 2-year",OUT,402.0
355,56,3080.0,"Public, 4-year or above",IN,772.0


In [8]:
df_temp

,ST_FIPS,sum,sector,status,count
0,1,32934.0,"Private, 4-year or above",IN,823.0
1,1,32934.0,"Private, 4-year or above",OUT,687.0
2,1,32934.0,"Public, 2-year",IN,10160.0
3,1,32934.0,"Public, 2-year",OUT,537.0
4,1,32934.0,"Public, 4-year or above",IN,12113.0
...,...,...,...,...,...
352,55,34182.0,"Public, 4-year or above",OUT,7907.0
353,56,3080.0,"Public, 2-year",IN,999.0
354,56,3080.0,"Public, 2-year",OUT,402.0
355,56,3080.0,"Public, 4-year or above",IN,772.0


In [9]:
# some function to extract key
# from dictionary
def get_key(val):
    for key, value in st_map.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [12]:
us_states = json.load(open('./usstates.json','r'))
for feature in us_states['features']:
    feature['id']=feature['properties']['STATE']

# Dash Board

In [13]:
# !pip install jupyter-dash
# !jupyter lab build
# !pip install dash_split_pane
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_split_pane

In [14]:
external_scripts = [
    'https://www.google-analytics.com/analytics.js',
    {'src': 'https://cdn.polyfill.io/v2/polyfill.min.js'},
    {
        'src': 'https://cdnjs.cloudflare.com/ajax/libs/lodash.js/4.17.10/lodash.core.js',
        'integrity': 'sha256-Qqd/EfdABZUcAxjOkMi8eGEivtdTkh3b65xCZL4qAQA=',
        'crossorigin': 'anonymous'
    }
]

# external CSS stylesheets
external_stylesheets = [
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

In [42]:
df = fe

app = JupyterDash(__name__,
                  external_scripts=external_scripts,
                  external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Freshmen Immigraion Analysis',
            style={"width": "100%",
                   "text-align": "center",
                   "padding-top": "5%",
                   "fontSize": 30}
           ),
    
    html.Br(),
    html.Br(),
    
    dash_split_pane.DashSplitPane(
        children=[
            # 1
            html.Div(children=[
                
            html.H1(children='Settings', style={'textAlign': 'center',
                                                'fontSize': 20}),
                
                html.Br(),
                html.Br(),
    
                dcc.Dropdown(
                    df['sector'].unique(),
                    id='student-education-sector',
                    placeholder="Select a education sector"
                ),
                
                html.Br(),
                
                dcc.RadioItems(
                    options=[
                        {'label': 'Stay in State', 'value': 'IN'},
                        {'label': 'Leave State', 'value': 'OUT'},
                    ],
                    value='IN',
                    id='student-status',
                    inline=True
                ),
                
                html.Br(),
                
                dcc.RadioItems(
                    options=[
                        {'label': 'Freshman', 'value': 'efres01'},
                        {'label': 'Freshman graduated within 12 month', 'value': 'efres02'},
                    ],
                    value='efres01',
                    id='highschool-graduation-time',
                    inline=True
                )
                
            ], style={'margin-left': '20%', 'verticalAlign': 'middle', 'font-size' : '120%'}),

            
            # 2
            html.Div(children=[
                
            html.H1(children='Graph View', style={'textAlign': 'center',
                                                  'fontSize': 20}),
                dcc.Graph(id='graph-with-slider'),
                dcc.Slider(
                    df['YEAR'].min(),
                    df['YEAR'].max(),
                    step=None,
                    value=df['YEAR'].min(),
                    marks={str(year): str(year) for year in df['YEAR'].unique()},
                    id='year-slider'
                )
            ])
        ],

        id="splitter",
        split="vertical",
        size=400,
    ) 
])

@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'),
    Input('student-status', 'value'),
    Input('student-education-sector', 'value'),
    Input('highschool-graduation-time', 'value')
)

def update_figure(selected_year,status,sector,gradtime):
    df_select = df[df.YEAR == selected_year]
    temp1 = df_select.groupby(['ST_FIPS']).sum()[gradtime].reset_index(name='sum')
    temp2 = df_select.groupby(['ST_FIPS','sector','status']).sum()[gradtime].reset_index(name='counts')

    df_temp = pd.merge(temp1,temp2,on='ST_FIPS')
    df_temp['ratio']=df_temp['counts']/df_temp['sum']
    df_temp.rename(columns={'ST_FIPS':'id'}, inplace=True)
    df_temp['state_name']=df_temp['id'].apply(lambda x: get_key(x))
    filtered_df = df_temp.loc[(df_temp.status==status) & (df_temp.sector==sector)]
    
    fig = px.choropleth_mapbox(filtered_df,locations='id',geojson=us_states,color='ratio',
                               # range_color=(0,1),
                               hover_name='state_name')
    fig.update_layout(mapbox_style="carto-positron",
                              mapbox_zoom=2.5, mapbox_center = {"lat": 38, "lon": -99})

    fig.update_layout(transition_duration=500)

    return fig


if __name__ == '__main__':
    app.run_server(mode='external',debug=True)

Dash app running on http://127.0.0.1:8050/


Without splitting the view

In [26]:
# df = fe

# app = JupyterDash(__name__,
#                   external_scripts=external_scripts,
#                   external_stylesheets=external_stylesheets)

# app.layout = html.Div(children=[
#     html.H1(children='Freshmen Immigraion Analysis',
#             style={"width": "100%",
#                    "text-align": "center",
#                    "padding-top": "5%",}
#            ),

# #     html.Div(children='''
# #         Dashboard
# #     '''),
    
#     html.Div([
#         html.Div(
#             dcc.Dropdown(
#                 df['sector'].unique(),
#                 id='student-education-sector',
#                 placeholder="Select a education sector"
#                 ), style={'width': '100%', 'display': 'inline-block'}
#         ),
#         html.Div(
#             dcc.RadioItems(
#                 options=[
#                     {'label': 'Stay in State', 'value': 'IN'},
#                     {'label': 'Leave State', 'value': 'OUT'},
#                 ],
#                 value='IN',
#                 id='student-status',
#                 inline=True
#             ), style={'width': '100%', 'display': 'inline-block'}
#         ),
#         html.Div(
#             dcc.RadioItems(
#                 options=[
#                     {'label': 'Freshman', 'value': 'efres01'},
#                     {'label': 'Freshman graduated within 12 month', 'value': 'efres02'},
#                 ],
#                 value='efres01',
#                 id='highschool-graduation-time',
#                 inline=True
#             ), style={'width': '100%', 'display': 'inline-block'}
#         )
#     ]),
    
#     dcc.Graph(id='graph-with-slider'),
#     dcc.Slider(
#         df['YEAR'].min(),
#         df['YEAR'].max(),
#         step=None,
#         value=df['YEAR'].min(),
#         marks={str(year): str(year) for year in df['YEAR'].unique()},
#         id='year-slider'
#     )
# ])

# @app.callback(
#     Output('graph-with-slider', 'figure'),
#     Input('year-slider', 'value'),
#     Input('student-status', 'value'),
#     Input('student-education-sector', 'value'),
#     Input('highschool-graduation-time', 'value')
# )

# def update_figure(selected_year,status,sector,gradtime):
#     df_select = df[df.YEAR == selected_year]
#     temp1 = df_select.groupby(['ST_FIPS']).sum()[gradtime].reset_index(name='sum')
#     temp2 = df_select.groupby(['ST_FIPS','sector','status']).sum()[gradtime].reset_index(name='counts')

#     df_temp = pd.merge(temp1,temp2,on='ST_FIPS')
#     df_temp['ratio']=df_temp['counts']/df_temp['sum']
#     df_temp.rename(columns={'ST_FIPS':'id'}, inplace=True)
#     df_temp['state_name']=df_temp['id'].apply(lambda x: get_key(x))
#     filtered_df = df_temp.loc[(df_temp.status==status) & (df_temp.sector==sector)]
    
#     fig = px.choropleth_mapbox(filtered_df,locations='id',geojson=us_states,color='ratio',
#                                # range_color=(0,1),
#                                hover_name='state_name')
#     fig.update_layout(mapbox_style="carto-positron",
#                               mapbox_zoom=2.5, mapbox_center = {"lat": 38, "lon": -99})

#     fig.update_layout(transition_duration=500)

#     return fig


# if __name__ == '__main__':
#     app.run_server(mode='external',debug=True)

Dash app running on http://127.0.0.1:8050/
